In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import pickle

Wmin = 0.001
Wmax = 0.25
Wnsteps = 30
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

Nmin = 10
Nmax = 150
Nnsteps = 30
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps

data = BPV.Data()
data.read_csv("p.delviva.csv",False)
data.df.sort_index(by="p",inplace=True,ascending=True)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
entropies = np.zeros((Wnsteps, Nnsteps))
rates = np.zeros((Wnsteps, Nnsteps))
cardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
indexes = {}

it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("Solving problem {0} out of {1}, for W = {2} and N = {3}".format(it, nproblems, W, N))
    
    prbl_exact = BPV.BPV("pulp",data,N,W,time_solver=False)
    prbl_exact.solve()
    #prbl_exact.pprint_solution()
    
    entropies[i,j] = prbl_exact.solution_entropy
    rates[i,j] = prbl_exact.solution_rate
    cardinalities[i,j] = prbl_exact.solution_cardinality
    indexes[(i,j)] = prbl_exact.solution.index
    it += 1
    
#f1 = open("/home/renato/tesi/code/plot6.entropies",r'wb')
#f2 = open("/home/renato/tesi/code/plot6.rates",r'wb')
#f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'wb')
#f4= open("/home/renato/tesi/code/plot6.indexes",r'wb')

#pickle.dump(entropies,f1)
#pickle.dump(rates,f2)
#pickle.dump(cardinalities,f3)
#pickle.dump(indexes,f4)


Solving problem 1 out of 900, for W = 0.001 and N = 10
Solving problem 2 out of 900, for W = 0.001 and N = 14
Solving problem 3 out of 900, for W = 0.001 and N = 18
Solving problem 4 out of 900, for W = 0.001 and N = 22
Solving problem 5 out of 900, for W = 0.001 and N = 26
Solving problem 6 out of 900, for W = 0.001 and N = 30
Solving problem 7 out of 900, for W = 0.001 and N = 34
Solving problem 8 out of 900, for W = 0.001 and N = 38
Solving problem 9 out of 900, for W = 0.001 and N = 42
Solving problem 10 out of 900, for W = 0.001 and N = 46
Solving problem 11 out of 900, for W = 0.001 and N = 50
Solving problem 12 out of 900, for W = 0.001 and N = 54
Solving problem 13 out of 900, for W = 0.001 and N = 58
Solving problem 14 out of 900, for W = 0.001 and N = 62
Solving problem 15 out of 900, for W = 0.001 and N = 66
Solving problem 16 out of 900, for W = 0.001 and N = 70
Solving problem 17 out of 900, for W = 0.001 and N = 74
Solving problem 18 out of 900, for W = 0.001 and N = 78
S

In [61]:
#SAVE
f1 = open("/home/renato/tesi/code/plot6.entropies",r'wb')
f2 = open("/home/renato/tesi/code/plot6.rates",r'wb')
f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'wb')
f4= open("/home/renato/tesi/code/plot6.indexes",r'wb')

pickle.dump(entropies,f1)
pickle.dump(rates,f2)
pickle.dump(cardinalities,f3)
pickle.dump(indexes,f4)


In [2]:
#LOAD
f1 = open("/home/renato/tesi/code/plot6.entropies",r'rb')
f2 = open("/home/renato/tesi/code/plot6.rates",r'rb')
f3 = open("/home/renato/tesi/code/plot6.cardinalities",r'rb')
f4 = open("/home/renato/tesi/code/plot6.indexes",r'rb')

entropies = pickle.load(f1)
rates = pickle.load(f2)
cardinalities = pickle.load(f3)
indexes = pickle.load(f4)

In [46]:
cm?

In [12]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
ax.plot_surface(X, Y, entropies, rstride=1, cstride=1, alpha=0.5, cmap=cm.hsv)
ax.plot_wireframe(X, Y, rates, rstride=1, cstride=1, alpha=0.8)
#ax.plot_wireframe(X, Y, cardinalities/150, rstride=1, cstride=1, alpha=0.8)

#cset = ax.contour(X, Y, entropies, zdir='z', offset=-100, cmap=cm.coolwarm)
#cset = ax.contour(X, Y, entropies, zdir='x',  cmap=cm.coolwarm)
#cset = ax.contour(X, Y, entropies, zdir='y',  cmap=cm.coolwarm)

ax.set_xlabel('W')
ax.set_ylabel('N')
ax.set_zlabel('Entropy')

plt.show()

In [9]:
zeroindexes = (entropies == 0).nonzero()
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
for i,j in zeroindexes:
    print(Wrange[i], Nrange[j])

0.0093 58
0.0093 62
0.0425 74
0.2168 26
0.2251 22
0.2251 26
0.2334 22
0.2334 46


In [5]:
prbl_exact = BPV.BPV("pulp",data,58,0.0093,time_solver=False)
prbl_exact.write_cplex_lp("/home/renato/tesi/code/bug")
#prbl_exact.solve()
#prbl_exact.pprint_solution()

prbl_exact = BPV.BPV("heuristic",data,58,0.0093,time_solver=False)
prbl_exact.solve()
prbl_exact.pprint_solution()


Solver =  heuristic 
Entropy =  0.0764990388237 
Cardinality =  43 
Rate =  0.0090633272987
